In [17]:
# Biblioteca
import os
import json
from googleapiclient.discovery import build
from datetime import datetime
import pandas as pd

In [18]:
# Variáveis locais
API_KEY = os.getenv("YOUTUBE_API_KEY")
CSV_PATH = "../config/videos_monitorados.csv"

In [19]:
# Conectando API do YouTube e fazendo a pasta de armazenamento
youtube = build('youtube', 'v3', developerKey=API_KEY)

coment_dir = "../data/comments"
os.makedirs(coment_dir, exist_ok=True)

In [20]:
# Salva os comentários em um JSON
df = pd.read_csv(CSV_PATH)
ids_ativos = df[df['status'] == 'ativo']['video_id'].tolist()

for video_id in ids_ativos:
    print(f"\n🧪 Coletando comentários do vídeo {video_id}")

    json_path = os.path.join(coment_dir, f"{video_id}.json")

    if os.path.exists(json_path):
        with open(json_path, "r", encoding="utf-8") as f:
            comentarios_salvos = json.load(f)
        comment_ids_existentes = {c['comment_id'] for c in comentarios_salvos}
    else:
        comentarios_salvos = []
        comment_ids_existentes = set()

    novos_comentarios = []
    next_page_token = None
    encontrou_existente = False

    while not encontrou_existente:
        response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=100,
            pageToken=next_page_token
        ).execute()

        for item in response.get("items", []):
            comment_id = item['id']
            comment = item['snippet']['topLevelComment']['snippet']

            if comment_id in comment_ids_existentes:
                print("⏹️ Comentário já salvo, interrompendo leitura.")
                encontrou_existente = True
                break

            novos_comentarios.append({
                "comment_id": comment_id,
                "author": comment.get('authorDisplayName'),
                "text": comment.get('textDisplay'),
                "publishedAt": comment.get('publishedAt'),
                "likeCount": comment.get('likeCount', 0)
            })

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    if novos_comentarios:
        print(f"💾 Salvando {len(novos_comentarios)} novos comentários.")
        todos = comentarios_salvos + novos_comentarios
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(todos, f, ensure_ascii=False, indent=2)
    else:
        print("✅ Nenhum comentário novo.")



🧪 Coletando comentários do vídeo po3iuig0aDY
⏹️ Comentário já salvo, interrompendo leitura.
✅ Nenhum comentário novo.

🧪 Coletando comentários do vídeo exqyJZQWsmw
⏹️ Comentário já salvo, interrompendo leitura.
✅ Nenhum comentário novo.

🧪 Coletando comentários do vídeo cncBTBNdmyw
💾 Salvando 1429 novos comentários.
